# Making a 2D N-Body Simulation

##### PHYS 512 Final Project

In [1]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import imageio
%matplotlib qt

#### Function to create the potential of a point charge

In [2]:
def point_charge_V(g, soft_r, b=5):
    x, y = np.meshgrid(np.linspace(-b,b, g+1), np.linspace(-b,b, g+1))
    r = np.sqrt(x**2+y**2)
    r[r<soft_r]=soft_r
    V = 1/r
    V=V[:-1,:-1]
    return V

The above function creates a softened potential for a point charge. The softening is done by setting a limit on the minimum distance "r", which is called "soft r". What this does essentially is that if a distance is less than the soft r, the potential at that point is equal to the potential at soft r. This makes it so the 1/r function doesn't go off to infinity at 0, but rather plateaus.

I decided to create the density function with 1 more gridcell than what will be input in the nbody class because I use even numbers for the number of gridcells, but if you create a point charge potential with an even number of gridcells, you don't obtain one central cell with maximum potential from which potential decreases with distance. Instead you get a 2x2 grid of max potential. This is a problem, because when the class will convolve the even point potential with the 2d density histogram, it will set one of the corners of the 2x2 cells on the cell for said point. When the gradient is then taken, the point will get affected by it's own potential, causing all the points to drift.

I could also just use an odd number of gridcells, but this was causing me issues (I think because of the fft and fftshift), so this was a solution that worked for me.

The difference between an even and odd number of gridcells for the point potential can be seen below:

<div>
<img src="images/other/even_point_V.png" width="400"><img src="images/other/odd_point_V.png" width="400">
</div>

#### The n-body class itself

In [2]:
class nbody:
    def __init__(self, n, gridcells, soft_r, x=False, m=False, v=False, periodic=True):
        self.n = n
        if x is False:
            self.x = np.random.rand(n,2)
        else:
            if x.shape[1]!=2:
                print('Position must be a Nx2 array')
                assert(1==0)
            elif int(n)!=len(x):
                print("Number of particles doesn't match number of initial positions.")
                assert(1==0)
            self.x = x
        if m is False:
            self.m = np.ones(n)
        else:
            if int(n)!=len(m):
                print("Number of particles doesn't match number of initial masses.")
                assert(1==0)
            self.m = m
        self.mstack = np.vstack((self.m,self.m)).T
        if v is False:
            self.v = np.zeros([n,2])
        else:
            if v.shape[1]!=2:
                print('Position must be a Nx2 array')
                assert(1==0)
            elif len(x)!=len(v):
                print("Number of initial positions doesn't match number of initial velocities.")
                assert(1==0)
            self.v = v
        self.g = gridcells
        self.soft = soft_r
        self.periodic = periodic
        self.pot = np.zeros([self.g,self.g])
        if periodic is True:
            self.point = point_charge_V(self.g, self.soft)
            self.pointft = np.fft.rfft2(self.point)
        else:
            self.point = point_charge_V(2*self.g, self.soft)            
            self.pointft = np.fft.rfft2(self.point)
            
        self.lostK = 0
        self.lostU = 0
            
    def density(self, rk4=False):
        if rk4 is False:
            density = np.histogram2d(self.x[:,0],self.x[:,1], bins=self.g, weights=self.m, range=[[0,1],[0,1]])[0]
        else:
            density = np.histogram2d(rk4[:,0],rk4[:,1], bins=self.g, weights=self.m, range=[[0,1],[0,1]])[0]

        if self.periodic is True:
            self.p = density
        else:
            self.p = np.zeros([self.g*2,self.g*2])
            self.p[self.g//2:3*self.g//2, self.g//2:3*self.g//2] = density
        return self.p
    
    def potential(self, rk4=False):
        if rk4 is False:
            density = self.density()
        else:
            density = self.density(rk4)
        
        densityft = np.fft.rfft2(density)
        
        if self.periodic is True:
            self.pot = np.fft.fftshift(np.fft.irfft2(densityft*self.pointft))
        else:
            big = np.fft.fftshift(np.fft.irfft2(densityft*self.pointft))
            self.pot = big[self.g//2:3*self.g//2, self.g//2:3*self.g//2]
            
        return self.pot
    
    def force(self, rk4=False):
        if rk4 is False:
            V = self.potential()
        else:
            V = self.potential(rk4)
        fx, fy = np.gradient(V)
        dx = 1/self.g
        dy = 1/self.g
        self.f = np.empty([len(self.x),2])
        for i in range(len(self.x)):
            indx, indy = int(self.x[i,0]/dx), int(self.x[i,1]/dy)
            if indx == self.g:
                indx=indx-1
            if indy == self.g:
                indy=indy-1
            self.f[i] = np.array([fx[indx,indy],fy[indx,indy]])
        return self.f
    
    def leapfrog(self, dt):
        self.x=self.x+dt*self.v
        if self.periodic is True:
            for i in self.x:
                if i[0]>1:
                    i[0]=i[0]-1
                if i[0]<0:
                    i[0]=i[0]+1
                if i[1]>1:
                    i[1]=i[1]-1
                if i[1]<0:
                    i[1]=i[1]+1
        else:
            problems = []
            for i in range(len(self.x)):
                if self.x[i,0]>1 or self.x[i,1]>1 or self.x[i,0]<0 or self.x[i,1]<0:
                    problems.append(i)
            if len(problems)!=0:
                for i in problems:
                    self.lostK = self.lostK + np.sum(0.5*self.mstack[i]*self.v[i]**2)
                    self.lostU = self.lostU + np.sum(self.pot)/self.n
                self.x = np.delete(self.x, problems, 0)
                self.v = np.delete(self.v, problems, 0)
                self.m = np.delete(self.m, problems)
                self.mstack = np.delete(self.mstack, problems, 0)
        self.v=self.v+self.force()*dt
        
    def get_derivs(self,xx):
        nn=xx.shape[0]//2
        x=xx[:nn,:]
        v=xx[nn:,:]
        f=self.force(x)
        return np.vstack([v,f])
    
    def rk4(self, dt):
        xv = np.vstack([self.x,self.v])
        k1 = self.get_derivs(xv)
        k2 = self.get_derivs(xv+k1*dt/2)
        k3 = self.get_derivs(xv+k2*dt/2)
        k4 = self.get_derivs(xv+k3*dt)
        
        tot=(k1+2*k2+2*k3+k4)/6
        v = tot[:self.n,:]
        f=tot[self.n:,:]
        
        self.x = self.x + v*dt
        if self.periodic is True:
            for i in self.x:
                if i[0]>1:
                    i[0]=i[0]-1
                if i[0]<0:
                    i[0]=i[0]+1
                if i[1]>1:
                    i[1]=i[1]-1
                if i[1]<0:
                    i[1]=i[1]+1
        else:
            problems = []
            for i in range(len(self.x)):
                if self.x[i,0]>1 or self.x[i,1]>1 or self.x[i,0]<0 or self.x[i,1]<0:
                    problems.append(i)
            if len(problems)!=0:
                for i in problems:
                    self.lostK = self.lostK + np.sum(0.5*self.mstack[i]*self.v[i]**2)
                    self.lostU = self.lostU + np.sum(self.pot)/self.n
                self.x = np.delete(self.x, problems, 0)
                self.v = np.delete(self.v, problems, 0)
                self.m = np.delete(self.m, problems)
                self.mstack = np.delete(self.mstack, problems, 0)
        self.v = self.v + f*dt

#### Explaining each function in the nbody class:
##### init:
The init function initializes the variables we will need for later. The nbody class itself takes in 7 parameters: the number of particles, the number of gridcells, the value for soft r, the initial position of each particle, the initial velocity of each particle, the mass of each particle and whether the boundary conditions are periodic or not. The initial postion, velocity and masses of each particle is an optional entry, and if none is specified, then x is created by generating n 2D random numbers between 0 and 1, v is set to 0 for each particle and m is set to 1 for each particle. Periodic is either true or false, if true, boundary conditions are periodic, if false, boundary conditions are non-periodic.

self.mstack is going to be used later for calculating kinetic energy, its just an nx2 array with each column being the mass vector.

self.pot is initialized so it can be called to plot the potential for the system later on. For now, the matrix is just set to 0.

At the end of the init function is the first divergence between the periodic and non-periodic boundary conditions. We want to generate our matrix for the point charge of a single particle, as well as the fft of said matrix to use later when correlating. For periodic boundary conditions, since the fourier transform itself is already a periodic function, we don't have to do any manipulations to our point charge potential. For non-periodic boundary conditions however, we have to make our point charge potential twice the size of the number of gricells. The reasoning for this will be explained below in the explanation for the density function.

In non-periodic boundary conditions, we also have to create 2 values to keep track of the kinetic and potential energy of lost particles, which is explained later in the leapfrog function.

##### density:
To create the density matrix, a 2D histogram is taken of the positions of all the particles. I chose to make the domain of my class [0,1] in both x and y, as it is easy to generate random numbers in this range using np.random.rand. The number of bins is set to the number of input gridcells, and the weights of each particle is set to the mass vector.

Here we see the first exception caused by our second step function: rk4. In leapfrog, we only ever need to find the density matrix of our particles, but in rk4, we need to take the density of values around the position of each particle, so we can't simply histogram self.x.

If we're using periodic boundary conditions, then our density matrix is done. If we're using non-periodic boundary conditions, we need to do a bit of padding.

So that the fft doesn't cause us periodic boundary conditions, we pad the density function with 0s in each direction equal in size to the original density matrix. This makes it so even when the fft wraps around, it will only see 0s on the other side, so the point won't feel a potential pulling it in that direction, which essentially makes it as if we cut it off and imposed a non-periodic system.

##### potential:
The potential function simply takes the density matrix and convolves it with the potential of a single point particle in order to obtain the potential of the entire system. The convolution is done by fourier transforming both the point particle potential and the density matrix, then multiplying them together and taking the inverse fourier transform.

Here again exceptions are needed if we use rk4 as our step.

If non-periodic boundary conditions are used, our potential matrix is 3 times the size we need it to be as it has been padded with 0s, so we need to pick out the area that will actually be needed from the padded potential.

##### force:
It's in the force function that the force on each particle is calculated. It receives as input the potential of the system and takes the gradient in both x and y. This gives us 2 grids with the force in x and y respectively on each gridcell, since force is equal to the gradient of potential. So now what we need to do is figure out in which gridcell each particle falls into. 

To do this, the total range (1 in this case) is divided by the number of gridcells in order to determine the size of each gridcell. The postition in x and y is then divided by this value to determine how many gridcells it takes to reach the particle. This in turn tells us the index in both x and y of the gridcell that a particle falls into. If the index is equal to the number of gridcells, the index is set to itself minus 1. This is done to include points falling on the outer boundary. For example, if you have 4 gridcells, then without this extra step, each gridcell would include particles falling in the following range: gc0=[0,0.25), gc1=[0.25,0.50), gc2=[0.50,0.75), gc3=[0.75,1). By adding this condition, we simply change it so now gc3=[0.75,1] to include particles falling on the top edge.

Once we know which gridcell each particle falls into, we set the force on that particle in x and y to the gradient in x and y of the potential at that same gridcell.

##### leapfrog:
Leapfrog is the function that actually takes a step. Leapfrog updates the position of each particle by adding v*dt to it. It then checks to make sure particles haven't fallen outside of our range after taking their step. Once that check has been made, it calculates the new velocity of each particle by adding f*dt to it.

If particles have fallen out of the range in periodic boundary conditions, then all that needs to be done is to subtract the x or y position (whichever one has left the range) by 1 (the length of our range) in order to slot it back in on the other side of the range.

If we are using non-periodic boundary conditions, things are trickier. Following conservation of energy, particles under non-periodic boundary conditions should never be able to escape the range, however since our grid doesn't have an infinite number of gridcells and our dt isn't extremely tiny, some particles do end up escaping. If particles do escape, they are simply removed from the system, and the kinetic and potential energy they had before they left is retained. To estimate the potential energy of the particle that just left the system, the total potential is divided by the number of particles.

##### rk4 and get_derivs:
get_derivs is lumped in with rk4 here as it is only used when using rk4 as our step. In rk4, we use the runga-kutta method to estimate the new position of each particle, rather than using leapfrog. This method calculates the force on a particle in positions near the postition of the particle, then averages them to estimate the force on the particle. Just like the leapfrog function, the new position is calculated by adding v*dt to itself.

The get_derivs function intakes an array of both position and velocity, finds the force on each position, and returns an array of velocity and force.

The new velocity for each particle is calculated using this force by adding f*dt to itself.

Similarly to the leapfrog function, the rk4 function also has exceptions for particles that leave the range with both periodic and non-periodic boundary conditions.

#### A function created for plotting the particles

In [4]:
#plotting function
def plots(particles, nsteps, dt, step='leapfrog', savedir=False, Vplots=False, trackE=False):
    if step != 'leapfrog' and step !='rk4':
        print('Only permitted step mehtods are "leapfrog" and "rk4"')
        assert(1==0)
    plt.figure(figsize=(6,6))
    if savedir is not False:
        filenames = []
    if trackE is not False:
        K = np.zeros(nsteps)
        U = np.zeros(nsteps)
    if step =='leapfrog' and Vplots is not True:
        for i in range(nsteps):
            if trackE is not False:
                K[i] = np.sum(0.5*particles.mstack*particles.v**2)+particles.lostK
                U[i] = np.sum(particles.potential(particles.x))+particles.lostU
            plt.clf()
            plt.plot(particles.x[:,0],particles.x[:,1],'.')
            plt.xlim(0,1)
            plt.ylim(0,1)
            plt.pause(0.001)
            particles.leapfrog(dt)
            if savedir is not False:
                plt.savefig('{}/lf_{}.png'.format(savedir,i))
                filenames.append('lf_{}.png'.format(i))
        if savedir is not False:
            images = []
            for file in filenames:
                img = cv2.imread(os.path.join(savedir,file))
                images.append(img)
            imageio.mimsave('{}/leapfrog.gif'.format(savedir), images)
        
    if step =='leapfrog' and Vplots is True:
        for i in range(nsteps):
            if trackE is not False:
                K[i] = np.sum(0.5*particles.mstack*particles.v**2)+particles.lostK
                U[i] = np.sum(particles.potential(particles.x))+particles.lostU
            plt.clf()
            plt.pcolormesh(particles.pot.T)
            plt.pause(0.001)
            particles.leapfrog(dt)
            if savedir is not False:
                plt.savefig('{}/lf_V_{}.png'.format(savedir,i))
                filenames.append('lf_V_{}.png'.format(i))
        if savedir is not False:
            images = []
            for file in filenames:
                img = cv2.imread(os.path.join(savedir,file))
                images.append(img)
            imageio.mimsave('{}/leapfrog_V.gif'.format(savedir), images)
    
    if step =='rk4' and Vplots is not True:
        for i in range(nsteps):
            if trackE is not False:
                K[i] = np.sum(0.5*particles.mstack*particles.v**2)+particles.lostK
                U[i] = np.sum(particles.potential(particles.x))+particles.lostU
            plt.clf()
            plt.plot(particles.x[:,0],particles.x[:,1],'.')
            plt.xlim(0,1)
            plt.ylim(0,1)
            plt.pause(0.001)
            particles.rk4(dt)
            if savedir is not False:
                plt.savefig('{}/rk4_{}.png'.format(savedir,i))
                filenames.append('rk4_{}.png'.format(i))
        if savedir is not False:
            images = []
            for file in filenames:
                img = cv2.imread(os.path.join(savedir,file))
                images.append(img)
            imageio.mimsave('{}/rk4.gif'.format(savedir), images)
    
    if step =='rk4' and Vplots is True:
        for i in range(nsteps):
            if trackE is not False:
                K[i] = np.sum(0.5*particles.mstack*particles.v**2)+particles.lostK
                U[i] = np.sum(particles.potential(particles.x))+particles.lostU
            plt.clf()
            plt.pcolormesh(particles.potential(particles.x).T)
            plt.pause(0.001)
            particles.rk4(dt)
            if savedir is not False:
                plt.savefig('{}/rk4_V_{}.png'.format(savedir,i))
                filenames.append('rk4_V_{}.png'.format(i))
        if savedir is not False:
            images = []
            for file in filenames:
                img = cv2.imread(os.path.join(savedir,file))
                images.append(img)
            imageio.mimsave('{}/rk4_V.gif'.format(savedir), images)
    if trackE is not False:
        return K, U, K+U

The above function takes in a class of particles, the number of steps we want to take, dt as well as which step function to use. There is also a parameter which allows you to save each plot in order to create a gif of the evolution of particle positions with time. The option to view plots of the potential of the system rather than the position of each particle as well as track the energy of the system is also possible using the extra parameters.

The function plots the position (or the potential of the system if that's what's selected) of each particle in the system, then takes a step (in either leapfrog or rk4 with a set dt), then plots the position again, then takes another step... and so on.

If the energy is wished to be tracked, then at each step the kinetic energy of the system is calculated using $K = \frac{1}{2}mv^2$ and the potential energy is calculated by summing up the total potential in the system. If non-periodic boundary conditions are being used, then the kinetic and potential energy of particles that have left the system has to be added to these values.

### Part 1:

In [13]:
parts = nbody(1, 200, 0.5, periodic=True)
plots(parts, 50, 0.005, step='leapfrog', savedir='images/particle_at_rest')

<div>
<img src="images/particle_at_rest/leapfrog.gif" width="400">
</div>

As can be seen in the super interesting gif above, a single starting at rest remains motionless.

### Part 2:

In [29]:
circlex = np.array([[0.4,0.5],[0.6,0.5]])
circlev = np.array([[0,0.05],[0,-0.05]])

parts=nbody(2,200,0.5, x=circlex, v=circlev, periodic=False)
plots(parts, 500, 0.05, step='leapfrog', savedir='images/rotate')

<div>
<img src="images/rotate/leapfrog.gif" width="400">
</div>

As you can see, two particles placed in a circular orbit continue to orbit eachother. And if you don't believe me that they follow a circular path, we can plot their trails too:

<div>
<img src="images/rotate_save/leapfrog.gif" width="400">
</div>

### Problem 3:

In [12]:
nparts = 500000
nsteps = 50
gridcells = 200
soft = 0.5
dt = 0.001

parts = nbody(nparts, gridcells, soft, periodic=False)
plots(parts, nsteps, dt, step='leapfrog', Vplots=False, trackE=False)

KeyboardInterrupt: 